In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep = '\t')
train_data.head()

In [ ]:
test_data = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep = '\t')
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
print("TF version: ", tf.__version__)
if tf.__version__ < "2.0.0":
    tf.enable_eager_execution()
    print("Eager execution enabled.")
else:
    print("Eager execution enabled by default.")

if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

In [ ]:
print(train_data.shape)
print(test_data.shape)


In [ ]:
train_data.columns

In [ ]:
test_data.columns

In [ ]:
print(train_data['Sentiment'].unique())
train_data['Sentiment'].nunique()

In [ ]:
train_data['Sentiment'].value_counts()

In [ ]:
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re

In [ ]:
def sentence_cleaning(df):
    sentence = []
    for sent in tqdm(df['Phrase']):
        
        text = re.sub("[^a-zA-Z]"," ",sent)
        
        word = word_tokenize(text.lower())
        
        lemmatizer = WordNetLemmatizer()
        
        lemm_word = [lemmatizer.lemmatize(i) for i in word]
        
        sentence.append(lemm_word)
    return (sentence)

In [ ]:
train_sent = sentence_cleaning(train_data)
test_sent = sentence_cleaning(test_data)

print(len(train_sent))
print(len(test_sent))

In [ ]:
from keras.utils import to_categorical
target_col = train_data.Sentiment.values
y_target = to_categorical(target_col)
y_target.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_sent,y_target,test_size = 0.2,stratify = y_target)

In [ ]:
unique_words = set()
max_len = 0
for sent in tqdm(X_train):
    unique_words.update(sent)
    if(max_len < len(sent)):
        max_len = len(sent)
        sentence = sent

In [ ]:
vocabulary = len(list(unique_words))
oov = '<OOV>'
embedding_dim = 300
padding = 'post'
trunc = 'post'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocabulary,oov_token = oov,char_level = False)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen = max_len,padding=padding,truncating = trunc)
X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val,maxlen = max_len,padding=padding,truncating = trunc)
X_test = tokenizer.texts_to_sequences(test_sent)
X_test = pad_sequences(X_test,maxlen = max_len,padding=padding,truncating = trunc)

In [ ]:
print(X_train.shape,X_test.shape,X_val.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Bidirectional,Activation,Dropout,LSTM,Embedding
from keras.layers.embeddings import Embedding

In [ ]:
model = tf.keras.Sequential()
model.add(Embedding(vocabulary,embedding_dim,input_length = max_len))
model.add(Bidirectional(LSTM(128, dropout = 0.8, recurrent_dropout=0.8, return_sequences=True)))
model.add(Bidirectional(LSTM(128,dropout = 0.5,recurrent_dropout=0.5,return_sequences=False)))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
hist_model = model.fit(X_train,y_train,
                    validation_data = (X_val, y_val),
                    epochs = 4,
                    batch_size = 256,
                    verbose = 1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist_model.history['accuracy'])
plt.plot(hist_model.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title('epochs vs accuracy')
plt.show()

In [ ]:
plt.plot(hist_model.history['loss'])
plt.plot(hist_model.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('epochs vs loss')
plt.legend()
plt.show()

In [ ]:
test_id = test_data['PhraseId']
test_id

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis = -1)
submission_df = pd.DataFrame({'PhraseId': test_id, 'Sentiment': y_pred})
submission_df.to_csv('submission.csv', index=False)
submission_df.head()